# Azure functions の例

このノートブックでは、Azure OpenAI サービスでの関数呼び出し機能の使用方法を示します。関数を使用することで、チャット補完の呼び出し元は、モデルが外部ツールやデータソースに機能を拡張するために使用できる機能を定義できます。

チャット関数の詳細については、OpenAI のブログをご覧ください：https://openai.com/blog/function-calling-and-other-api-updates

**注意**: チャット関数には、gpt-4 および gpt-35-turbo の `-0613` ラベルから始まるモデルバージョンが必要です。古いバージョンのモデルではサポートされていません。

## セットアップ

まず、必要な依存関係をインストールし、使用するライブラリをインポートします。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

### 認証

Azure OpenAI サービスは、APIキーとAzure Active Directoryトークン認証情報を含む複数の認証メカニズムをサポートしています。

In [2]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### API キーを使用した認証

OpenAI SDK を *Azure API キー* を使用するように設定するには、`api_key` をエンドポイントに関連付けられたキーに設定する必要があります（このキーは [Azure Portal](https://portal.azure.com) の *"リソース管理"* の下にある *"キーとエンドポイント"* で確認できます）。ここでリソースのエンドポイントも確認できます。

In [3]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### Azure Active Directoryを使用した認証
次に、Azure Active Directoryを使用して認証する方法を見てみましょう。まず、`azure-identity`ライブラリをインストールします。このライブラリは、認証に必要なトークン認証情報を提供し、`get_bearer_token_provider`ヘルパー関数を通じてトークン認証情報プロバイダーの構築を支援します。静的トークンを`AzureOpenAI`に提供するよりも`get_bearer_token_provider`を使用することが推奨されます。なぜなら、このAPIは自動的にトークンをキャッシュし、更新してくれるからです。

Azure OpenAIでAzure Active Directory認証を設定する方法の詳細については、[ドキュメント](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)を参照してください。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> 注意: AzureOpenAIは、以下の引数が提供されていない場合、対応する環境変数から推論します：

- `api_key` は `AZURE_OPENAI_API_KEY` から
- `azure_ad_token` は `AZURE_OPENAI_AD_TOKEN` から
- `api_version` は `OPENAI_API_VERSION` から
- `azure_endpoint` は `AZURE_OPENAI_ENDPOINT` から

## デプロイメント

このセクションでは、関数を呼び出すために使用できるGPTモデルのデプロイメントを作成します。

### デプロイメント: Azure OpenAI Studioで作成する
チャット補完で使用するモデルをデプロイしましょう。https://portal.azure.com にアクセスし、Azure OpenAIリソースを見つけて、Azure OpenAI Studioに移動します。「Deployments」タブをクリックし、チャット補完に使用したいモデルのデプロイメントを作成します。モデルに付けるデプロイメント名は、以下のコードで使用されます。

In [4]:
deployment = "" # Fill in the deployment name from the portal here

## 関数

セットアップと認証が完了したので、Azure OpenAIサービスで関数を使用できるようになりました。これはいくつかのステップに分かれています：

1. 関数を定義する
2. 関数定義をchat completions APIに渡す
3. レスポンスからの引数で関数を呼び出す
4. 関数のレスポンスをchat completions APIにフィードバックする

#### 1. 関数の定義

関数のリストを定義できます。各関数には、関数名、オプションの説明、および関数が受け取るパラメータ（JSONスキーマとして記述）が含まれます。

In [5]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location"],
        },
    }
]

#### 2. 関数定義をchat completions APIに渡す

これで関数をchat completions APIに渡すことができます。モデルが関数を呼び出すべきと判断した場合、choiceに`finish_reason`として"tool_calls"が設定され、どの関数を呼び出すかとその引数の詳細が`message`に含まれます。オプションで、`tool_choice`キーワード引数を設定して、モデルに特定の関数を強制的に呼び出させることもできます（例：`{"type": "function", "function": {"name": get_current_weather}}`）。デフォルトでは、これは`auto`に設定されており、モデルが関数を呼び出すかどうかを選択できるようになっています。

In [ ]:
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "What's the weather like today in Seattle?"}
]

chat_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)
print(chat_completion)

#### 3. レスポンスから引数を使って関数を呼び出す

関数呼び出しの名前は最初に提供されたもののいずれかとなり、引数には関数定義に含まれるスキーマに一致するJSONが含まれます。

In [ ]:
import json

def get_current_weather(request):
    """
    This function is for illustrative purposes.
    The location and unit should be used to determine weather
    instead of returning a hardcoded response.
    """
    location = request.get("location")
    unit = request.get("unit")
    return {"temperature": "22", "unit": "celsius", "description": "Sunny"}

function_call = chat_completion.choices[0].message.tool_calls[0].function
print(function_call.name)
print(function_call.arguments)

if function_call.name == "get_current_weather":
    response = get_current_weather(json.loads(function_call.arguments))

#### 4. 関数のレスポンスをchat completions APIにフィードバックする

関数からのレスポンスは、ロールを"function"に設定した新しいメッセージにシリアライズする必要があります。これで、モデルはレスポンスデータを使用して回答を作成します。

In [ ]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": json.dumps(response)
    }
)

function_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)

print(function_completion.choices[0].message.content.strip())